# Kaggle Titanic Classification Problem

## Imports

In [11]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.pyplot import subplots
from statsmodels.datasets import get_rdataset
import sklearn.model_selection as skm
from ISLP import load_data , confusion_table
from ISLP.models import ModelSpec as MS

from sklearn.tree import ( DecisionTreeClassifier as DTC ,
DecisionTreeRegressor as DTR ,
plot_tree ,
export_text)
from sklearn.metrics import (accuracy_score ,
log_loss)
from sklearn.ensemble import \
(RandomForestRegressor as RF ,
GradientBoostingRegressor as GBR)
from ISLP.bart import BART

## Boilerplate

In [17]:
df = pd.read_csv(filepath_or_buffer="../Data/train.csv")
test_df = pd.read_csv(filepath_or_buffer="../Data/test.csv")
X_without_dummies = df[["Pclass", "Sex", "Age", "SibSp", "Parch", "Fare", "Embarked"]]
X = pd.get_dummies(data = X_without_dummies)
Y = df[["Survived"]]

## EDA

First, we make a basic plot of each predictor 

## Analysis of the data

The focus of this notebook is primarily for me to practice trees so we start by analysing the data using a basic decision tree. This will also determine the most valuable predictors for our model. Note: Tree based methods are not scale sensitive so we will not be standardizing the data as according to the theory, it will not give us a useful performance increase. 

NameError: name 'fit_transform' is not defined